In [1]:
import os
import yfinance as yf
from langchain.tools import BaseTool
from typing import Any, Type
from pydantic import BaseModel, Field
#from langchain.agents import initialize_agent
from typing import Optional
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain.agents import tool
#from langchain.schema import SystemMessage
from langchain_community.tools.convert_to_openai import format_tool_to_openai_function
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from operator import itemgetter
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationEntityMemory
from langchain.chains import ConversationChain
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE



### Get Stock Price from yahoo finance - pydantic

In [2]:
class StockPriceCheckInput(BaseModel):
    """Input to check Stock price."""

    stock_ticker: str = Field(description="Ticker symbol for stock")
    
class StockPriceTool(BaseTool):
    name = "StockPriceTool"
    description = """
    Useful to find the stock price.
    """
    def _run(self, stock_ticker: str):
        ticker = yf.Ticker(ticker=stock_ticker)
        current_price = ticker.history(period="1d")
        return round(current_price['Close'].iloc[-1], 2)

    def _arun(self, stock_ticker: str):
        raise NotImplementedError()  
    
    args_schema: Type[BaseModel] = StockPriceCheckInput   

### Recommendations from Yahoo finance - pydantic 

In [3]:
class RecommendationStockInput(BaseModel):
    """Input to the stock recommendations"""
    recommend_stock_ticker: str = Field(description="Ticker symbol for stock")
    
class RecommendationStockTool(BaseTool):
    name = "RecommendationStockTool"
    description = "Useful to find recommendations to buy or hold if you have stocks,  if underperformed, not recommended. You should input the stock ticker which is used by yahoo finance API"
    
    def _run(self, recommend_stock_ticker: str):
     company = yf.Ticker(ticker=recommend_stock_ticker)
     return company.info['recommendationKey']

    def _arun(self, recommend_stock_ticker: str):
        raise NotImplementedError()  
    
    args_schema: Type[BaseModel] = RecommendationStockInput   

### Tools as openAI functions

In [7]:
stock_tools = [StockPriceTool(), RecommendationStockTool()]

### Model, Prompt, Agent and executor

In [12]:
from langchain.chains import LLMChain
llm = ChatOpenAI( api_key=os.environ['API_Key'],
                   temperature=0, max_retries=2).bind(functions = [format_tool_to_openai_function(f) for f in stock_tools])

#memory_llm = ChatOpenAI( api_key=os.environ['API_Key'],  model='gpt-3.5-turbo')

prompt = ChatPromptTemplate.from_messages([
    ("system", """
        You are a stock broker, help user's to get latest stock prices and recommendation to buy as of current date.
        You can also calculate the number of to buy or sell if user want to invest x amount of dollars.
     """),
    ("user", "{user_input}"),
    MessagesPlaceholder(variable_name = "agent_notepad")
])

# memory_buffer = ConversationBufferMemory( llm=memory_llm, memory_key="chat_history", 
#                           return_messages=True, output_key="output")

# conversation_with_memory = ConversationChain (
#     llm=memory_llm, 
#     verbose=True,
#     prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
#     memory=ConversationEntityMemory(llm=memory_llm, return_messages=True),
#     output_key="output"
# )

agent =({
        "user_input" : lambda x: x["input"],
        "agent_notepad": lambda x: format_to_openai_function_messages( x["intermediate_steps"]),
        #"chat_history": lambda x: x["chat_history"]
        }
        | prompt
        | llm
        | OpenAIFunctionsAgentOutputParser()
    )

open_ai_agent = AgentExecutor.from_agent_and_tools(agent=agent, 
                              tools=stock_tools, 
                              return_intermediate_steps=True, 
                              #memory=conversation_with_memory,
                              verbose=False, 
                              handle_parsing_errors=True)


### Get latest stock price

In [80]:
result_stock_price = open_ai_agent.invoke({"input" : "what is the latest price of Amazon stock"})

### Without tools

```
{'input': 'what is the latest price of Amazon stock',
 'output': 'To provide you with the latest price of Amazon stock, I need to access real-time data. However, I can guide you on how to check the current stock price yourself.\n\nYou can check the latest price of Amazon stock by visiting a financial website or using a stock market app. Some popular options include:\n\n1. Google Finance: Go to the Google Finance website and search for "Amazon stock price."\n2. Yahoo Finance: Visit the Yahoo Finance website and search for "Amazon stock price."\n3. Bloomberg: Access the Bloomberg website or app and search for "AMZN" (Amazon\'s stock ticker symbol) to find the current price.\n\nPlease note that stock prices can fluctuate throughout the trading day, so the price you see may not be the most up-to-date.',
 'intermediate_steps': []}
 ```

 ### With support of tools

 ```{'input': 'what is the latest price of Amazon stock',
 'output': 'The latest price of Amazon stock is $155.34.',
 'intermediate_steps': [(AgentActionMessageLog(tool='StockPriceTool', tool_input={'stock_ticker': 'AMZN'}, log="\nInvoking: `StockPriceTool` with `{'stock_ticker': 'AMZN'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'StockPriceTool', 'arguments': '{\n  "stock_ticker": "AMZN"\n}'}})]),
   155.34)]}```

In [81]:
result_stock_price

{'input': 'what is the latest price of Amazon stock',
 'chat_history': [HumanMessage(content='what is the latest price of Amazon stock'),
  AIMessage(content='The latest price of Amazon stock is $155.34.')],
 'output': 'The latest price of Amazon stock is $155.34.',
 'intermediate_steps': [(AgentActionMessageLog(tool='StockPriceTool', tool_input={'stock_ticker': 'AMZN'}, log="\nInvoking: `StockPriceTool` with `{'stock_ticker': 'AMZN'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'StockPriceTool', 'arguments': '{\n  "stock_ticker": "AMZN"\n}'}})]),
   155.34)]}

In [57]:
result_stock_price['output']

'The latest price of Amazon stock is $155.34.'

In [53]:
result_stock_price['intermediate_steps'][0]

(AgentActionMessageLog(tool='StockPriceTool', tool_input={'stock_ticker': 'AMZN'}, log="\nInvoking: `StockPriceTool` with `{'stock_ticker': 'AMZN'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'StockPriceTool', 'arguments': '{\n  "stock_ticker": "AMZN"\n}'}})]),
 154.62)

In [30]:
open_ai_agent.invoke({"input" : "is it recommended to buy Amazon stock ?"})

{'input': 'is it recommended to buy Amazon stock ?',
 'output': 'It is recommended to buy Amazon stock.',
 'intermediate_steps': [(AgentActionMessageLog(tool='RecommendationStockTool', tool_input={'recommend_stock_ticker': 'AMZN'}, log="\nInvoking: `RecommendationStockTool` with `{'recommend_stock_ticker': 'AMZN'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'RecommendationStockTool', 'arguments': '{\n  "recommend_stock_ticker": "AMZN"\n}'}})]),
   'buy')]}

In [82]:
open_ai_agent.invoke({"input" : "I have $2000, how many stocks I can buy?"})

{'input': 'I have $2000, how many stocks I can buy?',
 'chat_history': [HumanMessage(content='what is the latest price of Amazon stock'),
  AIMessage(content='The latest price of Amazon stock is $155.34.'),
  HumanMessage(content='I have $2000, how many stocks I can buy?'),
  AIMessage(content='To calculate the number of Amazon stocks you can buy with $2000, we need to know the current price of Amazon stock. Could you please confirm if the latest price of Amazon stock is $155.34?')],
 'output': 'To calculate the number of Amazon stocks you can buy with $2000, we need to know the current price of Amazon stock. Could you please confirm if the latest price of Amazon stock is $155.34?',
 'intermediate_steps': []}

### Get stock ticker.

In [13]:
open_ai_agent.invoke({"input" : "what is the latest price of Tesla stock"})

{'input': 'what is the latest price of Tesla stock',
 'output': 'The latest price of Tesla stock is $212.19.',
 'intermediate_steps': [(AgentActionMessageLog(tool='StockPriceTool', tool_input={'stock_ticker': 'TSLA'}, log="\nInvoking: `StockPriceTool` with `{'stock_ticker': 'TSLA'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'StockPriceTool', 'arguments': '{\n  "stock_ticker": "TSLA"\n}'}})]),
   212.19)]}

### Get recommendation from LLM to buy a stock.

In [104]:
open_ai_agent.invoke({"input":"what is the latest stock price of Beyond Meat, Inc, is it good to buy ?"})

{'input': 'what is the latest stock price of Beyond Meat, Inc, is it good to buy ?',
 'output': 'The latest stock price of Beyond Meat, Inc (BYND) is $8.44. According to the recommendation, the stock is currently underperforming. Therefore, it may not be a good time to buy this stock.',
 'intermediate_steps': [(AgentActionMessageLog(tool='StockPriceTool', tool_input={'stock_ticker': 'BYND'}, log="\nInvoking: `StockPriceTool` with `{'stock_ticker': 'BYND'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'StockPriceTool', 'arguments': '{\n  "stock_ticker": "BYND"\n}'}})]),
   8.44),
  (AgentActionMessageLog(tool='RecommendationStockTool', tool_input={'recommend_stock_ticker': 'BYND'}, log="\nInvoking: `RecommendationStockTool` with `{'recommend_stock_ticker': 'BYND'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'RecommendationStockTool', 'arguments': '{\n  "recommend_stock_ticker": "BYND"\n}'}})]),
   'u

### Number of stocks you can buy
Check to see if LLM can suggest the number of stocks to buy

In [14]:
open_ai_agent.invoke({"input":"I am planning to invest $5000 in apple stock, if recommended I want to buy this company stocks. If so how many stocks I can buy?"})

{'input': 'I am planning to invest $5000 in apple stock, if recommended I want to buy this company stocks. If so how many stocks I can buy?',
 'output': 'The current price of Apple stock is $181.18. To calculate how many stocks you can buy with $5000, divide the total amount by the stock price:\n\nNumber of stocks = $5000 / $181.18 = 27.59\n\nTherefore, you can buy approximately 27 stocks of Apple with $5000.',
 'intermediate_steps': [(AgentActionMessageLog(tool='StockPriceTool', tool_input={'stock_ticker': 'AAPL'}, log="\nInvoking: `StockPriceTool` with `{'stock_ticker': 'AAPL'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'StockPriceTool', 'arguments': '{\n  "stock_ticker": "AAPL"\n}'}})]),
   181.18)]}

In [13]:
open_ai_agent.invoke({"input":"I am planning to invest $5000 in apple stock, how many stocks can I buy and is it recommended to buy this company stocks?"})

{'input': 'I am planning to invest $5000 in apple stock, how many stocks can I buy and is it recommended to buy this company stocks?',
 'output': 'Based on the latest recommendations, it is recommended to buy Apple (AAPL) stock.',
 'intermediate_steps': [(AgentActionMessageLog(tool='StockPriceTool', tool_input={'stock_ticker': 'AAPL'}, log="\nInvoking: `StockPriceTool` with `{'stock_ticker': 'AAPL'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'StockPriceTool', 'arguments': '{\n  "stock_ticker": "AAPL"\n}'}})]),
   181.18),
  (AgentActionMessageLog(tool='RecommendationStockTool', tool_input={'recommend_stock_ticker': 'AAPL'}, log="\nInvoking: `RecommendationStockTool` with `{'recommend_stock_ticker': 'AAPL'}`\nresponded: The current price of Apple stock (AAPL) is $181.18. To calculate the number of stocks you can buy with $5000, divide the total amount by the stock price:\n\nNumber of stocks = $5000 / $181.18 = 27.59 (rounded down to the near

### Invest money and check how may stock to buy

In [108]:
open_ai_agent.invoke({"input" : "If I want to buy 25 Amazon stock, how much money I need to invest"})

{'input': 'If I want to buy 25 Amazon stock, how much money I need to invest',
 'output': 'The current price of Amazon stock is $145.24. If you want to buy 25 Amazon stocks, you would need to invest $3,631.',
 'intermediate_steps': [(AgentActionMessageLog(tool='StockPriceTool', tool_input={'stock_ticker': 'AMZN'}, log="\nInvoking: `StockPriceTool` with `{'stock_ticker': 'AMZN'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'StockPriceTool', 'arguments': '{\n  "stock_ticker": "AMZN"\n}'}})]),
   145.24)]}

In [52]:
print(agent_results)

Based on the recommendation, it is recommended to buy Apple (AAPL) stock. The current price of Apple stock is $193.58.

To calculate how much you need to invest for 25 stocks, you can multiply the stock price by the number of stocks:

Investment amount = Stock price * Number of stocks
Investment amount = $193.58 * 25
Investment amount = $4,839.50

Therefore, you would need to invest approximately $4,839.50 for 25 stocks of Apple.


### Try to buy underperformed stock

In [15]:
open_ai_agent.invoke({"input":"I am planning to invest $1000 in 'Beyond Meat, Inc', if recommended I want to buy this company stocks, else ignore it.?"})

{'input': "I am planning to invest $1000 in 'Beyond Meat, Inc', if recommended I want to buy this company stocks, else ignore it.?",
 'output': "Based on the recommendation, 'Beyond Meat, Inc' (BYND) is currently underperforming. Therefore, it is not recommended to buy stocks of this company at the moment.",
 'intermediate_steps': [(AgentActionMessageLog(tool='RecommendationStockTool', tool_input={'recommend_stock_ticker': 'BYND'}, log="\nInvoking: `RecommendationStockTool` with `{'recommend_stock_ticker': 'BYND'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'RecommendationStockTool', 'arguments': '{\n  "recommend_stock_ticker": "BYND"\n}'}})]),
   'underperform')]}

In [79]:
agent_results = open_ai_agent.invoke({"input" : "I am planning to invest $1000 in 'Beyond Meat, Inc' stock, is it recommended to invest in this stock, if so how many can I buy"})
agent_results

{'input': "I am planning to invest $1000 in 'Beyond Meat, Inc' stock, is it recommended to invest in this stock, if so how many can I buy",
 'output': 'Based on the current stock price of Beyond Meat, Inc (BYND) at $8.44, and the recommendation of "underperform", it is not recommended to invest in this stock. It is advisable to consider other investment options.',
 'intermediate_steps': [(AgentActionMessageLog(tool='StockPriceTool', tool_input={'stock_ticker': 'BYND'}, log="\nInvoking: `StockPriceTool` with `{'stock_ticker': 'BYND'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'StockPriceTool', 'arguments': '{\n  "stock_ticker": "BYND"\n}'}})]),
   8.44),
  (AgentActionMessageLog(tool='RecommendationStockTool', tool_input={'recommend_stock_ticker': 'BYND'}, log="\nInvoking: `RecommendationStockTool` with `{'recommend_stock_ticker': 'BYND'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'Recommendation

In [114]:
open_ai_agent.invoke({"input" : "I have 20 Tesla stocks, which I want to sell. How much money would I get back if i sell it today"})

{'input': 'I have 20 Tesla stocks, which I want to sell. How much money would I get back if i sell it today',
 'output': 'The current price of Tesla stock is $237.49. If you sell your 20 Tesla stocks today, you would get back $4,749.80.',
 'intermediate_steps': [(AgentActionMessageLog(tool='StockPriceTool', tool_input={'stock_ticker': 'TSLA'}, log="\nInvoking: `StockPriceTool` with `{'stock_ticker': 'TSLA'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'StockPriceTool', 'arguments': '{\n  "stock_ticker": "TSLA"\n}'}})]),
   237.49)]}

In [119]:
open_ai_agent.invoke({"input" : "I have 20 Tesla and Apple stocks, which I want to sell. How much money would I get back if i sell it today"})

{'input': 'I have 20 Tesla and Apple stocks, which I want to sell. How much money would I get back if i sell it today',
 'output': 'The current price of Tesla stock (TSLA) is $237.49 and the current price of Apple stock (AAPL) is $181.18. \n\nTo calculate how much money you would get back if you sell your 20 Tesla and Apple stocks, you need to multiply the number of stocks by their respective prices:\n\nFor Tesla:\n20 stocks * $237.49 = $4,749.80\n\nFor Apple:\n20 stocks * $181.18 = $3,623.60\n\nTherefore, if you sell your 20 Tesla and Apple stocks today, you would get back a total of $8,373.40.',
 'intermediate_steps': [(AgentActionMessageLog(tool='StockPriceTool', tool_input={'stock_ticker': 'TSLA'}, log="\nInvoking: `StockPriceTool` with `{'stock_ticker': 'TSLA'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'StockPriceTool', 'arguments': '{\n  "stock_ticker": "TSLA"\n}'}})]),
   237.49),
  (AgentActionMessageLog(tool='StockPriceTool', tool